# Data
## Data Overview
The [Radiological Society of North America](https://www.rsna.org/) (RSNA) has teamed with the American Society of Neuroradiology (ASNR) and the American Society of Spine Radiology (ASSR) to conduct an AI challenge competition designed to explore the extent to which artificial intelligence could be used to aid in the detection and localization of cervical spine fractures.

The challenge planning task force collected imaging data sourced from twelve sites on six continents, including approximately 3,000 CT studies. To provide ground-truth data, Spine radiology specialists from the ASNR and ASSR provided expert image level annotations for a these studies to indicate the presence, vertebral level and location of any cervical spine fractures. 

The full dataset, consisting of over 713,000 image, segmentation, and metadata files, spans some 340 GB. 

### Metadata Files
**train.csv**: Metadata for the train test set containing:     
- StudyInstanceUID - The study ID. There is one unique study ID for each patient scan.      
- patient_overall - One of the target columns. The patient level outcome, i.e. if any of the vertebrae are fractured.      
- C[1-7] - The other binary target columns which indicate whether the given vertebrae is fractured. 

**test.csv**: Metadata for the test set prediction structure. Only the first few rows of the test set are available for download.
- row_id - The row ID. This will match the same column in the sample submission file.      
- StudyInstanceUID - The study ID.               
- prediction_type - Which one of the eight target columns needs a prediction in this row.       

### Image Files
**[train/test]_images/[StudyInstanceUID]/[slice_number].dcm**: The image data are in the Digital Imaging and Communications in Medicine (DICOM) format, and are organized in one folder per StudyInstanceUID, e.g. patient. Each folder contains the image or slices which make up the CT Scan Study.

**sample_submission.csv**: A valid sample submission.  
- row_id - The row ID. See the test.csv for what prediction needs to be filed in that row.
- fractured - The target column.

**train_bounding_boxes.csv**: Bounding boxes for a subset of the training set.

### Segmentations
**segmentations/** Pixel level annotations for a subset of the training set. This data is provided in the Neuroimaging Informatics Technology Initiative (NIfTI) file format.  A portion of the imaging datasets have been segmented automatically using a 3D UNET model, and radiologists modified and approved the segmentations. The provided segmentation labels have values of 1 to 7 for C1 to C7 (seven cervical vertebrae) and 8 to 19 for T1 to T12 (twelve thoracic vertebrae are located in the center of your upper and middle back), and 0 for everything else. As we focused on the cervical spine, all scans have C1 to C7 labels but not all thoracic labels. 

The image data have been downloaded from the Kaggle site using the Kaggle API.

## Data Staging
To make the data available to Colab and other cloud-based runtime environments, the RSNA 2022 Cervical Spine Fracture Detection Dataset will be extracted, transformed and loaded (ETL) into a Google Drive. Our objective is to produce the following space-efficient, analysis and training-ready datasets.


| # | File / Directory Name                     | format   | Description                                                                                                              |
|---|-------------------------------------------|----------|--------------------------------------------------------------------------------------------------------------------------|
| 1 | segmentation_images/[StudyInstanceUID]    | TFRecord | Directory of batched TFRecords, each containing the images and   segmentation targets for an individual study.           |
| 2 | segmentation_metadata.csv                 | csv      | File containing StudyInstanceUID, SliceNumber, and the associated   Segmentation Vertebrae appearing in the Slice.       |
| 3 | train_bounding_boxes.csv                  | csv      | File containing the bounding boxes for each fracture annotated in the   segmentation files.                              |
| 4 | detection_train_images/[StudyInstanceUID] | TFRecord | Directory of batched TFRecords, each containing the images and fracture   targets for an individual study.               |
| 5 | detection_test_images[StudyInstanceUID]   | TFRecord | Directory containing a single TFRecord batch of three test images.                                                       |
| 6 | detection_train_metadata.csv              | csv      | File containing the metadata extracted from the DICOM files along with   the target fracture data in the train.csv file. |

Our ETL workflow is organized into a configuration, a pipeline object, and a set of operations. Starting with the configuration, the end-to-end process is defined as follows: